In [283]:
'''
Converts all strings to UTF-8
Removes HTML tags
Replaces returns ('\r' or '\n') with spaces.
Renames/reports duplicate colums
Removes/reports blank rows
Removes/reports duplicate header rows
''' 

import csv
from HTMLParser import HTMLParser
inFile = 'fileName.csv'
outFile = '.csv'.join(inFile.split('.csv')[:-1])+'-fixed.csv'
blankRows, headerRows = 0, 0

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ''.join(self.fed)

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

def cleanRow(row):
    # remove HTML and replace line feeds
    return [strip_tags(column).replace('\r', ' ').replace('\n', ' ') for column in row]

with open(inFile,'rUb') as fin, open (outFile,'wb') as fout:
    while(True):
        header = next(csv.reader(fin))
        #skip blank rows above header
        if sum(len(c) for c in header) > 0:
            break
        blankRows += 1
    
    newHeader = cleanRow(header)[:]
    colNames = dict.fromkeys(header, [])
    for i, n in enumerate(header):
        if colNames[n]:
            newHeader[i] = '{} ({})'.format(n, len(colNames[n]))
            print 'Renamed duplicate column {} to {}'.format(n, newHeader[i])

        colNames[n] = colNames[n] + [n]
    writer = csv.writer(fout)
    writer.writerow(newHeader)

    for n, row in enumerate(csv.reader(fin)):
        rowOk = True
        cleanRow(row)

        #skip row if blank
        if not sum(len(c) for c in row) > 0:
            blankRows += 1
            rowOk = False

        #skip duplicate header rows
        if row == header:
            headerRows += 1
            rowOk = False

        #do not write skipped rows
        if rowOk:
            try:
                writer.writerow(row)
            except UnicodeEncodeError:
                writer.writerow([s.encode('utf-8') for s in row])
        
if headerRows > 0:
    print headerRows, 'Duplicate header rows cleaned:'

if blankRows > 0:
    print blankRows, 'Blank rows cleaned:'

print 'output file is:', outFile


output file is: fileName-fixed.csv
